<a href="https://colab.research.google.com/github/ahreumcho/ahl/blob/main_project/exploration/ahreum_arxiv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting 

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install arxiv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import pandas as pd

In [ ]:
# Run this cell to mount your drive (you will be prompted to sign in)
from google.colab import drive
drive.mount('/content/drive')

!mkdir ~/.kaggle

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

1) ArXiv Data Wranggling

In [ ]:
!kaggle datasets download -d Cornell-University/arxiv
!unzip /content/arxiv.zip

100% 1.14G/1.14G [00:07<00:00, 189MB/s]
100% 1.14G/1.14G [00:07<00:00, 157MB/s]
Archive:  /content/arxiv.zip
  inflating: arxiv-metadata-oai-snapshot.json  


In [ ]:
chunk_size = 100000
chunks = []

for chunk in pd.read_json('arxiv-metadata-oai-snapshot.json', lines=True, chunksize=chunk_size):
    chunks.append(chunk)

arxiv_df = pd.concat(chunks, ignore_index=True)


**Arxiv Data**
* id: ArXiv ID (can be used to access the paper, see below)
* submitter: Who submitted the paper
* authors: Authors of the paper
* title: Title of the paper
* comments: Additional info, such as number of pages and figures
* journal-ref: Information about the journal the paper was published in
* doi: [https://www.doi.org](Digital Object Identifier)
* abstract: The abstract of the paper
* categories: Categories / tags in the ArXiv system
* versions: A version history



In [ ]:
arxiv_df.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [ ]:
import requests
from lxml import etree

url = "https://arxiv.org/category_taxonomy"

response = requests.get(url)
categories = pd.DataFrame(columns=['cat_id', 'main_category','sub_category'])
if response.status_code == 200:
    parser = etree.HTMLParser()
    tree = etree.fromstring(response.content, parser=parser)
    
    for x, y in zip(tree.xpath("//*[@id='category_taxonomy_list']//h4/text()"), tree.xpath("//*[@id='category_taxonomy_list']//h4/span/text()")):
      cat_id = x.strip()
      sub_cat = y.strip("()").strip()
      big_cat = cat_id.split(".")[0]
      categories = pd.concat([categories, pd.DataFrame({'cat_id': [cat_id], 'main_category' : [big_cat], 'sub_category': [sub_cat]})], ignore_index=True)
    
    big_category = tree.xpath("//*[@id='category_taxonomy_list']/h2/text()") + tree.xpath("//*[@id='category_taxonomy_list']//h3/text()")
categories

,cat_id,main_category,sub_category
0,cs.AI,cs,Artificial Intelligence
1,cs.AR,cs,Hardware Architecture
2,cs.CC,cs,Computational Complexity
3,cs.CE,cs,"Computational Engineering, Finance, and Science"
4,cs.CG,cs,Computational Geometry
...,...,...,...
150,stat.CO,stat,Computation
151,stat.ME,stat,Methodology
152,stat.ML,stat,Machine Learning
153,stat.OT,stat,Other Statistics


In [ ]:
categories['main_category'].unique()

array(['cs', 'econ', 'eess', 'math', 'astro-ph', 'cond-mat', 'gr-qc',
       'hep-ex', 'hep-lat', 'hep-ph', 'hep-th', 'math-ph', 'nlin',
       'nucl-ex', 'nucl-th', 'physics', 'quant-ph', 'q-bio', 'q-fin',
       'stat'], dtype=object)

In [ ]:
big_category

['Computer Science',
 'Economics',
 'Electrical Engineering and Systems Science',
 'Mathematics',
 'Physics',
 'Quantitative Biology',
 'Quantitative Finance',
 'Statistics',
 'Astrophysics',
 'Condensed Matter',
 'General Relativity and Quantum Cosmology',
 'High Energy Physics - Experiment',
 'High Energy Physics - Lattice',
 'High Energy Physics - Phenomenology',
 'High Energy Physics - Theory',
 'Mathematical Physics',
 'Nonlinear Sciences',
 'Nuclear Experiment',
 'Nuclear Theory',
 'Physics',
 'Quantum Physics']

In [ ]:
cat_map_dict = {}
cat_map_dict['cs'] = 'Computer Science'
cat_map_dict['econ'] = 'Economics'
cat_map_dict['eess'] = 'Electrical Engineering and Systems Science'
cat_map_dict['math'] =  'Mathematics'
cat_map_dict['astro-ph'] = 'Astrophysics'
cat_map_dict['cond-mat'] = 'Condensed Matter'
cat_map_dict['gr-qc'] = 'General Relativity and Quantum Cosmology'
cat_map_dict['hep-ex'] =  'High Energy Physics - Experiment'
cat_map_dict['hep-lat'] = 'High Energy Physics - Lattice'
cat_map_dict['hep-ph'] =  'High Energy Physics - Phenomenology'
cat_map_dict['hep-th'] = 'High Energy Physics - Theory'
cat_map_dict['math-ph'] = 'Mathematical Physics'
cat_map_dict['nlin'] = 'Nonlinear Sciences'
cat_map_dict['nucl-ex'] = 'Nuclear Experiment'
cat_map_dict['nucl-th'] =  'Nuclear Theory'
cat_map_dict['physics'] = 'Physics'
cat_map_dict['quant-ph'] = 'Quantum Physics'
cat_map_dict['q-bio'] = 'Quantitative Biology'
cat_map_dict['q-fin'] = 'Quantitative Finance'
cat_map_dict['stat'] = 'Statistics'

In [ ]:
categories['main_category'] = categories['main_category'].apply(lambda x : cat_map_dict[x])

In [ ]:
categories

,cat_id,main_category,sub_category
0,cs.AI,Computer Science,Artificial Intelligence
1,cs.AR,Computer Science,Hardware Architecture
2,cs.CC,Computer Science,Computational Complexity
3,cs.CE,Computer Science,"Computational Engineering, Finance, and Science"
4,cs.CG,Computer Science,Computational Geometry
...,...,...,...
150,stat.CO,Statistics,Computation
151,stat.ME,Statistics,Methodology
152,stat.ML,Statistics,Machine Learning
153,stat.OT,Statistics,Other Statistics


In [ ]:
# categories merge

# make categories list first

In [ ]:
arxiv_df['authors'].value_counts().head(100)

CMS Collaboration                            1169
ATLAS Collaboration                          1071
ALICE Collaboration                           426
The BABAR Collaboration, B. Aubert, et al     263
Saharon Shelah                                224
                                             ... 
Costantino Sigismondi                          67
Kiran S. Kedlaya                               67
Igor Nikolaev                                  67
Carlo Rovelli                                  67
Boaz Tsaban                                    67
Name: authors, Length: 100, dtype: int64

In [ ]:
arxiv_df['categories'].value_counts().head(100)

astro-ph                             86911
hep-ph                               78753
quant-ph                             63397
hep-th                               56969
cs.CV                                44160
                                     ...  
cond-mat.soft cond-mat.stat-mech      3343
physics.soc-ph                        3332
math.OA                               3316
cs.CL cs.LG                           3252
cond-mat.mes-hall cond-mat.str-el     3225
Name: categories, Length: 100, dtype: int64

ModuleNotFoundError: ignored